### Important librairies

In [53]:
#Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from lxml import objectify
%matplotlib inline

#Convert country name to ISO-3
import country_converter as coco

#World map plot librairies
import plotly.offline as po
import plotly.graph_objs as pg

# Analysis of the lexical fields
from empath import Empath 

#Warnings
import warnings

data_folder = './data/'

## 1) Data handling

### Import of the dataset


#### Movie metadata

In [2]:
### Movie metadata import
df_movie = pd.read_table(data_folder +'movie.metadata.tsv',header=None)
df_movie.columns=['Wikipedia_movie_ID' , 'Freebase_movie_ID', 'Movie_name' , 'Movie_date' , 'Movie_revenue' , 'Movie_runtime' , 'Movie_languages' , 'Movie_countries' , 'Movie_genres']
df_movie['Movie_countries'] = df_movie['Movie_countries'].apply( lambda x: list(eval(x).values()))
df_movie['Movie_genres'] = df_movie['Movie_genres'].apply( lambda x: list(eval(x).values()) )


#Movie metadata of american movies
df_movie_usa = df_movie.loc[df_movie['Movie_countries'].explode().isin(['United States of America'])[df_movie['Movie_countries'].explode().isin(['United States of America'])].index]
df_movie_usa

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi..."
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Musical, Comedy, Black-and-white]"
10,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Drama, Black-and-white]"
...,...,...,...,...,...,...,...,...,...
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","[France, United States of America]","[Parody, Americana, Comedy]"
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},[United States of America],"[Thriller, Erotic thriller, Psychological thri..."
81727,24209227,/m/07k5mlk,"The Time, the Place and the Girl",1946,NaN,105.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Comedy film, Romance Film, Musical]"
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],[Drama]


#### Character metadata

In [3]:
### Character metadata import
df_character = pd.read_table(data_folder +'character.metadata.tsv',header=None)
df_character.columns=['Wikipedia_movie_ID' , 'Freebase_movie_ID' , 'Movie_date' , 'Character_name' , 'Actor_date_of_birth' , 'Actor_gender' , 'Actor_height' , 'Actor_ethnicity' , 'Actor_name' , 'Actor_age_at_movie_release' , 'Freebase_character_actor_ID' , 'Freebase_character_ID' , 'Freebase_actor_ID'  ]

#Correct negative or too high actor age
df_character['Actor_age_at_movie_release']=df_character['Actor_age_at_movie_release'].apply(lambda x: -x if x<0 else x)
df_character['Actor_age_at_movie_release']=df_character['Actor_age_at_movie_release'].apply(lambda x: float("nan") if x>130 else x)

#Character metadata of american movies
df_character_usa = df_character.merge(df_movie_usa['Freebase_movie_ID'], on='Freebase_movie_ID')
df_character_usa.to_csv('df_character_usa.csv')

#### Movie summary

In [4]:
#Dataframe with existing movie summary
wiki_id_list = []
for ctr,wiki_id in enumerate(list(df_movie_usa['Wikipedia_movie_ID'])) : 
    path = data_folder+'/corenlp_plot_summaries/'+str(wiki_id)+'.xml'
    if os.path.exists(path):
        wiki_id_list.append(wiki_id)
wiki_id_series = pd.Series(wiki_id_list, name='Wikipedia_movie_ID',dtype=object)

#Data frame (movie metadata) with only existing summaries 
df_movie_usa_summary=df_movie_usa.merge(wiki_id_series, on='Wikipedia_movie_ID')


#Data frame (character metadata) with only existing summaries
df_character_usa_summary=df_character_usa.merge(wiki_id_series, on='Wikipedia_movie_ID')


In [5]:
#Summary exploration
df=df_movie_usa_summary

#Initialization
lexicon = Empath()
df_lex_fields = pd.DataFrame()
dic_lex_fields=dict.fromkeys(df['Wikipedia_movie_ID'].values,[])

for ctr,movie in enumerate(list(df['Wikipedia_movie_ID'].values)) :

    path = data_folder+'/corenlp_plot_summaries/'+str(movie)+'.xml'
    if os.path.exists(path):
        #Extract dataframe from xml file and convert it into a dataframe
        df_summary=pd.DataFrame()
        df_summary = pd.read_xml(data_folder+'/corenlp_plot_summaries/'+str(movie)+'.xml' , xpath='//token',parser='lxml')
        df_summary.rename(columns={'id':'word_id'},inplace=True)
        df_summary.insert(0, "sentence_id",df_summary['word_id'].ne(df_summary['word_id'].shift()+1).cumsum(), True) #add sentence id by indexing sequences  
        
        ###Lexical field analysis
        summary_words=list(df_summary['word'].values) #all words of the summary in a list
        summary_words=[str(word) for word in summary_words] #convert all words to a string
        features_lex_fields = pd.Series([lexicon.analyze(list(summary_words), categories = ["feminine","sexist","sexiest","beauty","beautiful","positive_emotion","negative_emotion"])])
        #Store data into a dictionnary
        dic_lex_fields[movie]= features_lex_fields[0]
        #Add a column corresponding to the number of words in the summary
        dic_lex_fields[movie]['Number_of_words']=len(df_summary)
        
        
        
        ###Count of he/she        
        #Filter only the pronouns
        df_summary_pronouns= df_summary[df_summary.POS=='PRP']
        #Count the number of redundant words 
        personal_pronouns = df_summary_pronouns.groupby(['word'])['word'].count().sort_values(ascending=False)

        if 'he' in list(personal_pronouns.index):
            dic_lex_fields[movie]['he_count'] = personal_pronouns['he']
        else:
            dic_lex_fields[movie]['he_count'] = 0

        if 'she' in list(personal_pronouns.index):
            dic_lex_fields[movie]['she_count'] = personal_pronouns['she'] 
        else:
            dic_lex_fields[movie]['she_count'] = 0
        
        
        
        ###Principal characters according to the summary
        characters_name = df_summary[df_summary['NER'] == 'PERSON']['word'].value_counts().index
        if len(characters_name)>=2:
            dic_lex_fields[movie]['Principal_summary_character'] =  characters_name[0] 
            dic_lex_fields[movie]['Secondary_summary_character'] =  characters_name[1] 
        if len(characters_name)==1:
            dic_lex_fields[movie]['Principal_summary_character'] =  characters_name[0] 
            dic_lex_fields[movie]['Secondary_summary_character'] =  float('nan')    
        if len(characters_name)==0:
            dic_lex_fields[movie]['Principal_summary_character'] =  float('nan')
            dic_lex_fields[movie]['Secondary_summary_character'] =  float('nan')   

df_lex_fields = pd.DataFrame.from_dict(dic_lex_fields, orient='index')  
#df_lex_fields.insert(loc=0, column='Movie_name', value=df['Movie_name'].values)
df_lex_fields.insert(loc=1, column='Freebase_movie_ID', value=df['Freebase_movie_ID'].values)
df_usa_summary_processed=df_movie_usa_summary.merge(df_lex_fields, on='Freebase_movie_ID')
df_usa_summary_processed


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,feminine,...,sexiest,beauty,beautiful,positive_emotion,negative_emotion,Number_of_words,he_count,she_count,Principal_summary_character,Secondary_summary_character
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",1.0,...,0.0,0.0,0.0,0.0,7.0,396,1,0,Ballard,Williams
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Romantic comedy, Ensemble Film, Comedy-drama,...",3.0,...,0.0,0.0,0.0,2.0,2.0,257,2,5,Adam,Kate
2,11250635,/m/02r52hc,The Mechanical Monsters,NaN,NaN,NaN,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Science Fiction, Adventure, Animation, Short ...",1.0,...,0.0,1.0,0.0,0.0,3.0,707,4,2,Lois,Clark
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Children's/Family, Musical, Fantasy, Comedy, ...",1.0,...,0.0,1.0,0.0,8.0,5.0,1678,12,7,Mary,Banks
4,21926710,/m/05p45cv,White on Rice,2009,NaN,82.0,{},[United States of America],"[Romantic comedy, Romance Film, Comedy, Indie]",1.0,...,0.0,1.0,0.0,2.0,0.0,125,3,0,Jimmy,Tak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20784,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100.0,"{""/m/02h40lc"": ""English Language""}",[United States of America],"[Alien Film, Science Fiction, Family Film, Com...",0.0,...,0.0,0.0,0.0,2.0,4.0,639,1,0,Bipto,Wrenchmuller
20785,26044505,/m/0b6m67n,The Flying Serpent,1946,NaN,59.0,{},[United States of America],"[Thriller, B-movie, Horror]",0.0,...,0.0,0.0,0.0,0.0,3.0,37,1,0,Andrew,Forbes
20786,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","[France, United States of America]","[Parody, Americana, Comedy]",1.0,...,0.0,0.0,0.0,2.0,1.0,252,1,1,Carla,Walt
20787,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},[United States of America],"[Thriller, Erotic thriller, Psychological thri...",1.0,...,0.0,0.0,0.0,1.0,9.0,611,6,8,Greenhill,Haines


In [6]:
#df=df_movie_usa.merge(df_usa_summary_processed, on=['Wikipedia_movie_ID','Freebase_movie_ID','Movie_name','Movie_date','Movie_revenue','Movie_runtime','Movie_languages','Movie_countries','Movie_genres'], how='outer')
df=df_movie_usa.merge(df_usa_summary_processed, on=['Wikipedia_movie_ID','Freebase_movie_ID','Movie_name','Movie_date','Movie_revenue','Movie_runtime','Movie_languages'], how='outer')
df.drop('Movie_genres_y', inplace=True, axis=1)
df.drop('Movie_countries_y', inplace=True, axis=1)
df.rename(columns={"Movie_genres_x": "Movie_genres", "Movie_countries_x": "Movie_countries"},inplace=True)

#Export final dataframe for movie metadata
df.to_csv('df_movie_usa.csv')


#### Import CSV

In [7]:
df_movie_usa=pd.read_csv('df_movie_usa.csv')
df_character_usa=pd.read_csv('df_character_usa.csv')

## 2) Focus on the USA

In [ ]:
#Initialisation of country converter to iso-3
cc = coco.CountryConverter()

#Occurences of movies for each country
countries = df_movie['Movie_countries'].explode().value_counts() #count the number of occurences
df_countries = countries.to_frame(name='Number_of_movies')
df_countries.reset_index(inplace=True)
df_countries = df_countries.rename(columns = {'index':'Countries'})
df_countries['percentage']=(df_countries['Number_of_movies']/df_countries['Number_of_movies'].sum())*100
df_countries['percentage']=df_countries['percentage'].apply(lambda x: int(x))

It is important to notice that for each movies, several countries name are specified. We consider each of these countries as country of origin of the movie. Thus, the total number of occurences is much larger than the total number of movies 

In [57]:

#Convert country name
some_countries = df_countries['Countries']
iso3_codes = cc.pandas_convert(series=some_countries, to='ISO3')   
df_countries['code']=iso3_codes


#Plot 
data = dict(type='choropleth', 
           locations = df_countries['code'], 
           z = df_countries['percentage'], 
           text = df_countries['Countries'])

layout = dict(title = 'Percentage of movies in the dataset - robinson projection',
              geo = dict( projection = {'type':'robinson'},
                         showlakes = False, 
                         lakecolor = 'rgb(0,191,255)'))
x = pg.Figure(data = [data], 
              layout = layout)
po.iplot(x)

West Germany not found in regex
Soviet Union not found in regex
German Democratic Republic not found in regex
Yugoslavia not found in regex
England not found in regex
Weimar Republic not found in regex
Scotland not found in regex
Socialist Federal Republic of Yugoslavia not found in regex
Serbia and Montenegro not found in regex
Federal Republic of Yugoslavia not found in regex
Wales not found in regex
Northern Ireland not found in regex
Soviet occupation zone not found in regex
Malayalam Language not found in regex
Crime not found in regex
German Language not found in regex
Ukranian SSR not found in regex


We see unequivocally that the main country of origin of the movies is the USA with 39% of all the occurences. The second main country is India with 9% of all the occurences. This great difference leads us to focus our study on the USA. In fact, we want to see a time evolution of the representation of women. From this, it is important to restrict the area of study to one country so that we remove the differences in the history of the different countries. Furthemore, as the history of the USA is well documented, we could try to link our results to importants events or social phenomena.

## 3) Data processing

### 3.1) Numeric data processing

### 3.2) Lexical data processing

### 3.3) Google trends